# con Anchors:

En cada una de estos hay 5 corriendo sucesivas

In [1]:
import itertools
import pickle
import matplotlib.pyplot as plt
import numpy as np
import math
import random
import os
import time
from numba import jit, njit
from numba.types import bool_, int_, float32
from math import comb
from copy import deepcopy
from tqdm import tqdm
import networkx as nx
import pandas as pd
from collections import defaultdict
from itertools import permutations

In [2]:
d_store = pickle.load(open("Drosophila_Hemispheres_allign_sensorydivided.pickle", "rb"))
R_groups = d_store["Right"]
L_groups = d_store["Left"] 

R_label = d_store["R_labels"] 
L_label = d_store["L_labels"] 

## Con los datos que teníamos R_groups, L_groups serían los M's// tablas == 2

In [3]:
tablas = 2
rows, columns = L_groups.shape[0] , R_groups.shape[0]
M = np.zeros((tablas,rows,columns))

M[0,:,:] = R_groups.copy()
M[1,:,:] = L_groups.copy()


## No son cuadradas, pero puede que a mí me interese que lo sean (y llenar lo que sobre de ceros)
M_square = np.zeros((tablas, rows, rows))
M_square[:,:, 0:columns] = M[:,:,:]


#Lo queremos BINARIO, ignorando su peso (Luego pensar cómo se haría con el peso)
M_bin = np.zeros((tablas,rows,columns))
M_square_bin = np.zeros((tablas,rows,rows))
for i in range(tablas):
    for j in range(rows):
        for k in range(columns):
            if (M[i,j,k] >= 1):
                M_bin[i,j,k] = 1
                M_square_bin[i,j,k] = 1
                


## FUNCIONES DE ENERGÍA

In [4]:
@jit(nopython=True)
def overlap_total_prob(L_f, A_f, P_inv_f):
    Nx = L_f.shape[0]
    Ny = L_f.shape[1]
    K = A_f.shape[0]
    
    ovlp_0 = np.zeros((K))
    ovlp_1 = np.zeros((K))
    for k in range(0,K):
        for f in range(0,Ny): #Overlap de cada fila (pero no lo necesitamos guardar, solo queremos el total)
            for c in range(0,Nx):
            #El overlap depende de la permutación!
                p_f=int(P_inv_f[k,f])
                p_c=int(P_inv_f[k,c])  
                
                valor_L, valor_A = L_f[f,c], A_f[k,p_f,p_c]
                
                ovlp_0[k] = ovlp_0[k] + (1-valor_L)*(1-valor_A )#+ (1-valor_L)*valor_A
                ovlp_1[k] = ovlp_1[k] + valor_L*valor_A #+ (1-valor_A)*valor_L
                
                
    ovlp_1 = int(sum(ovlp_1))
    ovlp_0 = int(sum(ovlp_0))
   
   
    return ovlp_0, ovlp_1






@jit(nopython = True)
def hamiltonian_prob(Edges_NoL, Edges_L, overlap_0, overlap_1, alpha, beta):
    #gamma(o11 + alpha)*gamma(o10 + beta)/(gamma(sum_E + alpha + beta))
    A_1 = overlap_1 + alpha
    B_1 = (Edges_L - overlap_1 + beta)
    C_1 = Edges_L + alpha + beta
    
    A_0 = overlap_0 + alpha
    B_0 = (Edges_NoL - overlap_0 + beta)
    C_0 = Edges_NoL + alpha + beta
    
    # !!!! Con numba los factoriales grandes dan errores: usar [ math.lgamma(n+1) == log(n!) ]
    H1 = math.lgamma(A_1)+ math.lgamma(B_1) - math.lgamma(C_1) 
    H0 = math.lgamma(A_0)+ math.lgamma(B_0) - math.lgamma(C_0) 
    
    H = -(H1 + H0)

    return H
    
    


@jit(nopython=True)
def L_wiring(A_f, P_inv_f):
    
    Nx = A_f.shape[1]
    Ny = A_f.shape[2]
    K = A_f.shape[0]
    L_new_f = np.zeros((Nx,Ny))
    
    for i in range(0,Nx):
        for j in range(0,Ny):
            for k in range(0,K):
                #Necesito la permutación
                p1 = int(P_inv_f[k,i]) #A la fila y columna i de L, la fila y columna que le corresponde en A
                p2 = int(P_inv_f[k,j]) #A la fila y columna j de L, la fila y columna que le corresponde en A
                L_new_f[i,j] += A_f[k,p1,p2]
            
#             print('i,j', i,j,'L_new', L_new_)
            valor_lnew=1/K* L_new_f[i,j]
            L_new_f[i,j] = round( valor_lnew )

    return L_new_f


#Este tiene en cuenta una lista secundaria
#(que puede ser por ejemplo el hub de mayor tamaño)
@jit(nopython=True)
def partition(array,  etiquetas, begin, end):
    pivot = begin
    for i in range(begin+1, end+1):
        if array[i] < array[begin]:
            pivot += 1
            array[i], array[pivot] = array[pivot], array[i]
            etiquetas[i], etiquetas[pivot] = etiquetas[pivot], etiquetas[i]
    array[pivot], array[begin] = array[begin], array[pivot]
    etiquetas[pivot], etiquetas[begin] = etiquetas[begin], etiquetas[pivot] 

    return pivot
@jit(nopython=True)
def quicksort(array, etiquetas, begin=0, end=None):
    #print('dentro:',array, begin, end)
    if end is None:
        end = len(array) - 1
    if begin >= end: #Para acabar el algoritmo
        return
    pivot = partition(array,  etiquetas, begin, end)
    #print('pivot', pivot, 'array', array)
    
    #Aquí ordenamos la derecha y la izquierda
    quicksort(array, etiquetas, begin, pivot-1)
    quicksort(array,  etiquetas, pivot+1, end)
    
@njit 
#Vamos a ordenar igual las etiquetas de nodos con las de a que grupo pertenecen
def permu_nogroups(L_f, A_f): #En este caso no lo voy a hacer con anchors
    Nx = L_f.shape[0]
    Ny = L_f.shape[1]
    K = A_f.shape[0]
    
    
    #Matrices permutación y sus inversas
    P_f = np.zeros((K,Nx))
    P_inv_f = np.zeros((K,Nx))
    P_new = np.zeros((K,Nx))
    
    #Vamos a ordenar las matrices Observación según el grado de los nodos de L_inf
    orden_L = np.zeros((Nx))
    for i in range(Nx):
        b = np.nonzero(L_f[i,:])
        orden_L[i] = b[0].size#.size
        
    array_L = np.arange(Nx)
    quicksort(orden_L, array_L)
    #Luego para igualar el array_L con el array_A tenemos que tener en cuenta lo de labels_f
    

        
    #Órdenes de A
    for i in range(0,K):
        orden_A = np.zeros((Nx))
        for i_orden in range(Nx):
            b = np.nonzero(A_f[i,i_orden,:])
            orden_A[i_orden] = b[0].size

        array_A = np.arange(Nx) #Para las etiquetas
        quicksort(orden_A, array_A)
            
        array_A_labels = array_A[np.argsort(array_L)]
        P_f[i,:] = array_A_labels
        
        for i_inv in range(0,Nx):
            for j_inv in range(0,Ny):
                if (P_f[i,i_inv] == j_inv):
                    P_inv_f[i, j_inv] = i_inv
    
    P_todo = np.zeros((2,K,Nx))
    P_todo[0,:,:] = P_f.copy()
    P_todo[1,:,:] = P_inv_f.copy()

    
    
    return P_todo
#Vamos a ordenar igual las etiquetas de nodos con las de a que grupo pertenecen
def permu_groups_capa1_anchors_knwn(L_f, A_f, start_f,grupos_t, Anchors,grupos_extra): #En este caso no lo voy a hacer con anchors
   
    Nx = L_f.shape[0]
    Ny = L_f.shape[1]
    K = A_f.shape[0]
    N_groups = len(start_f)
    start_2 = np.zeros((len(start_f)))
    start_2[1:] = np.cumsum(grupos_extra)[:-1]
    start_anchor = start_f - start_2   
    start_anchor = start_anchor.astype(int)
    
    #Matrices permutación y sus inversas
    P_f = np.zeros((K,Nx))
    P_inv_f = np.zeros((K,Nx))
    P_new = np.zeros((K,Nx))
    
    
    ### Colocamos primero los anchors
    n_an = len(Anchors)
    P_f[:,:n_an] = Anchors
    
    #Vamos a ordenar las matrices Observación según el grado de los nodos de L_inf
    orden_L_in = np.zeros((Nx-n_an ))
    array_L = np.zeros((Nx-n_an ))
    c_o = 0

    for i in range(Nx):
        if (i not in Anchors):
            b = np.nonzero(L_f[i,:])
            array_L[c_o] = i
            orden_L_in[c_o] = b[0].size
            c_o += 1
     
    orden_L = orden_L_in.copy()
    quicksort(orden_L, array_L)
    array_L = array_L[::-1]

    grupos_L = np.zeros((Nx-n_an))
    for  i_g in range(N_groups):
        start = start_anchor[i_g]
        if (i_g != N_groups-1):
            end = start_anchor[i_g+1]
        else:
            end = Nx-n_an
        grupos_L[start:end] = i_g
    grupos_L = grupos_L.astype(int)
    orden_L = orden_L_in.copy()
    quicksort(orden_L, grupos_L )
    grupos_L = grupos_L[::-1]
    print('array_L', np.sort(array_L)[:5])

    #Órdenes de A
    for i in range(0,K):
        orden_A_in = np.zeros((Nx- n_an))
        array_A = np.zeros((Nx-n_an))
        grupos_A = np.zeros((Nx-n_an))
        c_o = 0
        for i_orden in range(Nx):
            if (i_orden not in Anchors):
                b = np.nonzero(A_f[i,i_orden,:])
                array_A[c_o] = i_orden
                grupos_A[c_o] = grupos_t[i, i_orden]
                orden_A_in[c_o] = b[0].size#.size
                c_o += 1
        
        orden_A = orden_A_in.copy()
        quicksort(orden_A, array_A)  
        array_A = array_A[::-1]
        print('array_A', np.sort(array_A)[:5])

        orden_A = orden_A_in.copy()
        quicksort(orden_A, grupos_A)
        grupos_A = grupos_A[::-1]

        
        array_A_L = np.zeros((Nx - n_an))
        for i_n in range(Nx-n_an):
            if ((grupos_A[i_n] == grupos_L[i_n])):
                array_A_L[i_n] = array_A[i_n]
            else:
                # Busco el siguiente del mismo grupo y lo cambio
                count = 0
                for i_nn in range(i_n, Nx-n_an):
                    if ((grupos_A[i_nn] == grupos_L[i_n])): 
                        array_A_L[i_n] = array_A[i_nn]
                        #Cambio
                        array_A[i_nn] = array_A[i_n]
                        aux = grupos_A[i_nn]
                        grupos_A[i_nn] = grupos_A[i_n]
                        grupos_A[i_n] = aux
                        count = 1
                        break
        array_A_L = array_A_L[np.argsort(array_L)]
        P_f[i,n_an:] = array_A_L
        
        for i_inv in range(0,Nx):
            for j_inv in range(0,Ny):
                if (P_f[i,i_inv] == j_inv):
                    P_inv_f[i, j_inv]=i_inv
    
    P_todo = np.zeros((2,K,Nx))
    P_todo[0,:,:] = P_f.copy()
    P_todo[1,:,:] = P_inv_f.copy()

        

    
    
    return P_todo

# CONDICIONES INICIALES Y GRUPOS

Ahora es cuando vamos a poner los unkwons!

In [5]:
start_groups = d_store["start_groups"]
end_groups = d_store["end_groups"] 
size_groups = d_store["size_groups"]
names_groups = d_store["name_groups"] 

n_groups = len(size_groups)

print('grupos:',names_groups)
print('SIZE GROUPS:', size_groups)
#Para evitar trabajar con diccionarios (por NUMBA), vamos a hacer matrices 
# Cada matriz tiene el tamaño de la de mayor, pero luego las vamos cortando
rows_g, columns_g = size_groups.max(), size_groups.max()
G = np.zeros((tablas, n_groups, rows_g, columns_g))  #Aunque en las matrices G no voy a estar muy interesada
labels = np.zeros((rows))

print('Primer nodo de cada grupo', start_groups)
print('Último nodo de cada grupo', end_groups)

grupos: ['visual' 'olfactory' 'thermo-warm' 'gustatory-external'
 'gustatory-pharyngeal' 'thermo-cold' 'gut' 'respiratory' 'LN' 'PN'
 'DN-SEZ' 'ascending' 'pre-DN-VNC' 'PN-somato' 'pre-DN-SEZ' 'DN-VNC' 'RGN'
 'LHN' 'KC' 'CN' 'MB-FFN' 'MB-FBN' 'MBON' 'MBIN']
SIZE GROUPS: [  0  21   2  65  53   3  40  13  54 103  82  23 238  76  51  91  27 101
  23  50  27  54  24  14]
Primer nodo de cada grupo [   0    0   21   23   88  141  144  184  197  251  354  436  459  697
  773  824  915  942 1043 1066 1116 1143 1197 1221]
Último nodo de cada grupo [   0   21   23   88  141  144  184  197  251  354  436  459  697  773
  824  915  942 1043 1066 1116 1143 1197 1221 1235]


In [6]:
### DATOS
Nx, Ny = rows, rows #Nodos en la red

K = 2 #tablas #Matrices copiadas K
alpha,beta = 5,2 #Distribución beta que usamos
Edges = Nx*Ny #Entradas de la matriz de adjacency
micropasos = Edges*K #Para pasar por todas las posibilidades

np.random.seed(np.random.randint(199))
A = np.zeros((K,Nx,Ny))
A = M_square_bin[:, :,:]
                
print('MATRICES OBSERVACIÓN')  
print(A.shape)

####################################
## CONDICIONES INICIALES ##
L_ini = np.zeros((Nx,Ny))
P_ini_0 = np.zeros((K,Nx))
P_inv_ini_0 = np.zeros((K,Nx))
L_ini = (A[0,:,:]).copy()

groups_ini = np.zeros((K,Nx))
for m1 in range(K):
    for i_g in range(n_groups):
        start = start_groups[i_g]
        end = end_groups[i_g]
        groups_ini[m1, start:end] = i_g

MATRICES OBSERVACIÓN
(2, 1235, 1235)


# Anchors

In [7]:
# L_anchors, R_anchors = L_pos_anchors.copy(), R_pos_anchors.copy()
start_anchor = d_store["start_anchors"] 
end_anchor = d_store["end_anchor"] 
grupos_extra = d_store["anchors_group"] 
Anchors = d_store["anchors_all"] 

groups_ini_anchor = np.zeros((K,Nx))-1
for m1 in range(K):
    for i_g in range(n_groups):
        start = start_anchor[i_g]
        end = end_anchor[i_g]
        groups_ini_anchor[m1, start:end] = i_g


In [8]:
grupos_extra

array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  21.,  21.,  34.,
         0., 113.,  32.,  20.,  16.,   3.,  28.,   1.,   3.,   0.,   0.,
         0.,   0.])

In [9]:
#Vamos a ordenar igual las etiquetas de nodos con las de a que grupo pertenecen
def permu_groups_capa1_anchors_knwn(L_f, A_f, start_f,end_f,start_anch_f,end_anch_f,grupos_t, Anchors,grupos_extra): #En este caso no lo voy a hacer con anchors
   
    Nx = L_f.shape[0]
    Ny = L_f.shape[1]
    K = A_f.shape[0]
    N_groups = len(start_f)
    
    
    #Matrices permutación y sus inversas
    P_f = np.zeros((K,Nx))
    P_inv_f = np.zeros((K,Nx))
    P_new = np.zeros((K,Nx))
    
    
    ### Colocamos primero los anchors
    n_an = len(Anchors)
    P_inv_f[:,:n_an] = Anchors
    
    #Vamos a ordenar las matrices Observación según el grado de los nodos de L_inf
    orden_L_in = np.zeros((Nx-n_an ))
    array_L = np.zeros((Nx-n_an ))
    c_o = 0

    for i in range(Nx):
        if (i not in Anchors):
            b = np.nonzero(L_f[i,:])
            array_L[c_o] = i
            orden_L_in[c_o] = b[0].size
            c_o += 1
     
    orden_L = orden_L_in.copy()
    quicksort(orden_L, array_L)
    array_L = array_L[::-1]

    grupos_L = np.zeros((Nx-n_an))
    c_o = 0
    for  i_g in range(N_groups):
        for i_x in range(start_f[i_g], end_f[i_g]):
            if i_x not in Anchors:
                grupos_L[c_o] = i_g
                c_o += 1
    grupos_L = grupos_L.astype(int)
    orden_L = orden_L_in.copy()
    quicksort(orden_L, grupos_L )
    grupos_L = grupos_L[::-1]
    print('array_L', np.sort(array_L)[:5])

    #Órdenes de A
    for i in range(0,K):
        orden_A_in = np.zeros((Nx- n_an))
        array_A = np.zeros((Nx-n_an))
        grupos_A = np.zeros((Nx-n_an))
        c_o = 0
        for i_orden in range(Nx):
            if (i_orden not in Anchors):
                b = np.nonzero(A_f[i,i_orden,:])
                array_A[c_o] = i_orden
                grupos_A[c_o] = grupos_t[i, i_orden]
                orden_A_in[c_o] = b[0].size#.size
                c_o += 1
        
        orden_A = orden_A_in.copy()
        quicksort(orden_A, array_A)  
        array_A = array_A[::-1]
        print('array_A', np.sort(array_A)[:5])

        orden_A = orden_A_in.copy()
        quicksort(orden_A, grupos_A)
        grupos_A = grupos_A[::-1]

        
        array_A_L = np.zeros((Nx - n_an))
        for i_n in range(Nx-n_an):
            if ((grupos_A[i_n] == grupos_L[i_n])):
                array_A_L[i_n] = array_A[i_n]
            else:
                # Busco el siguiente del mismo grupo y lo cambio
                count = 0
                for i_nn in range(i_n, Nx-n_an):
                    if ((grupos_A[i_nn] == grupos_L[i_n])): 
                        array_A_L[i_n] = array_A[i_nn]
                        #Cambio
                        array_A[i_nn] = array_A[i_n]
                        aux = grupos_A[i_nn]
                        grupos_A[i_nn] = grupos_A[i_n]
                        grupos_A[i_n] = aux
                        count = 1
                        break
        array_A_L = array_A_L[np.argsort(array_L)]
        P_inv_f[i,n_an:] = array_A_L
        
        for i_inv in range(0,Nx):
            for j_inv in range(0,Ny):
                if (P_inv_f[i,i_inv] == j_inv):
                    P_f[i, j_inv]=i_inv
    
    P_todo = np.zeros((2,K,Nx))
    P_todo[0,:,:] = P_f.copy()
    P_todo[1,:,:] = P_inv_f.copy()

        

    
    
    return P_todo

In [10]:
P_inis = permu_groups_capa1_anchors_knwn(L_ini, A, start_groups,end_groups,start_anchor,end_anchor,groups_ini, Anchors, grupos_extra) 
P_inis = P_inis.astype(int)
P_ini_0 = (P_inis[0,:,:]).copy()#
P_inv_ini_0 = (P_inis[1,:,:]).copy()
print('*************************')
print(P_ini_0)
print(P_inv_ini_0)

array_L [0. 1. 2. 3. 4.]
array_A [0. 1. 2. 3. 4.]
array_A [0. 1. 2. 3. 4.]
*************************
[[ 292  293  294 ... 1232 1233 1234]
 [ 309  295  297 ... 1229 1230 1224]]
[[ 611  789  480 ... 1232 1233 1234]
 [ 611  789  480 ... 1229 1221 1228]]


In [11]:
for i_x in range(Nx):
    valor = P_inv_ini_0[1,P_ini_0[1,i_x]]
    if (i_x != valor):
        print('mal hecho jeje')
#print('start ini:', start_groups)
for i_g in range(n_groups):
    start = start_anchor[i_g]
    end = end_anchor[i_g]    
    for i_x in range(start, end):
        if ((P_inv_ini_0[0,i_x] < start_groups[i_g]) or (P_inv_ini_0[0,i_x] > end_groups[i_g]) ):
            print('mal hecho fila 0 en i_x', i_x)
        if ((P_inv_ini_0[1,i_x] < start_groups[i_g]) or (P_inv_ini_0[1,i_x] > end_groups[i_g]) ):
            print('mal hecho fila 1 en i_x', i_x)
        


In [12]:
###  VOY A PONER UN VECTOR DE UNKWONS de Nx 
## ( y otro de entradas== K-1 o buscar np.where == K-1, por ahora busco pero no sé )
# En cada entrada pongo el número de capas con groups == 6
# cuando hago los cambios en los que se incluyen un unkwons resto y sumo. 
# Si alguno llega a ==  K-1 ---> y hay más de dos ---> propongo el cambio y veo si lo acepta

## Temperaturas



In [13]:
# LISTA DE TEMPERATURAS
beta_o = 1.03
total = 17
b_exp = np.linspace(-50,0,num = total)
temper = beta_o**b_exp
tempers = temper#np.zeros((total+1))
# for i_1 in range(len(temper)):
#     if (temper[i_1]<1 and temper[i_1+1]>1):
#         tempers[:i_1+1] = temper[:i_1+1]
#         tempers[i_1+1] = 1.0000
#         tempers[i_1+1+1:] = temper[i_1+1:]
#         break

total_i = total
tempers_odd1 = np.arange(0,total_i-1,2, dtype = int)
tempers_odd2 = np.arange(1, total_i, 2,dtype = int)
tempers_odd = np.zeros((2, len(tempers_odd1)))
tempers_odd[0,:] = tempers_odd1
tempers_odd[1,:] = tempers_odd2

tempers_even1 = np.arange(1,total_i,2)
tempers_even2 = np.arange(2, total_i, 2)
tempers_even = np.zeros((2, len(tempers_even1)))
tempers_even[0,:] = tempers_even1
tempers_even[1,:] = tempers_even2

print('Odd', tempers_odd, tempers_even)


print('betas:', tempers)
N_t = len(tempers)
        
P_ini_t = np.zeros((N_t,K,Nx))
P_inv_ini_t = np.zeros((N_t,K,Nx))
L_ini_t = np.zeros((N_t,Nx,Ny)) 
groups_ini_t = np.zeros((N_t, K, Nx ))
groups_anch_ini_t = np.zeros((N_t, K, Nx ))

for i in range(0,N_t): #Aquí estoy inicializando para las temperaturas
    P_ini_t[i,:,:] = P_ini_0[:,:].copy()
    P_inv_ini_t[i,:,:] = P_inv_ini_0[:,:].copy()
    L_ini_t[i,:,:] = L_ini.copy()
    groups_ini_t[i,:,:] = groups_ini.copy()
    groups_anch_ini_t[i,:,:] = groups_ini_anchor.copy()

Odd [[ 0.  2.  4.  6.  8. 10. 12. 14.]
 [ 1.  3.  5.  7.  9. 11. 13. 15.]] [[ 1.  3.  5.  7.  9. 11. 13. 15.]
 [ 2.  4.  6.  8. 10. 12. 14. 16.]]
betas: [0.22810708 0.25018144 0.27439198 0.30094543 0.3300685  0.36200986
 0.39704225 0.4354648  0.47760557 0.52382438 0.57451588 0.63011289
 0.69109013 0.75796826 0.83131831 0.91176659 1.        ]


# Funciones MonteCarlo: Parallel sampling

In [14]:
@jit(nopython=True)
def overlap_total_change_filas(m1,L_f_v1v2_old,L_c_v1v2_old,L_f_v1v2,L_c_v1v2,A_f, P_inv_old, i_change, j_change,pp1, pp2): 

    Nx = L_f_v1v2_old.shape[1]
    K = A_f.shape[0]
    
    ovlp_0_new , ovlp_1_new = 0,0
    ovlp_0_old , ovlp_1_old = 0,0
    ovlp_0_dif, ovlp_1_dif = 0,0
  
    #old and new: changes and mapping changes
    changes = np.array([i_change, j_change], dtype = np.int32)
    changes_p = np.array([pp1, pp2], dtype = np.int32)  
    changes_old = np.array([j_change, i_change], dtype = np.int32)
    changes_p_old = np.array([pp2, pp1], dtype = np.int32)
    
    for i_k in range(K):
        for i_chan,change in enumerate(changes):
            for i_x in range(Nx):
                if (i_k != m1): # Only changes in the networks m1
                    p_f = int(P_inv_old[i_k,change])
                    p_c = int(P_inv_old[i_k,i_x])
                    p_old, p_old_c = p_f, p_c
                
                else:
                    p_f = changes_p[i_chan]
                    p_old = changes_p_old[i_chan]
                    if (i_x == i_change):
                        p_c = pp1
                        p_old_c = pp2
                    elif(i_x == j_change):
                        p_c = pp2
                        p_old_c = pp1
                    else: # Only changes in the nodes i,j
                        p_c = int(P_inv_old[i_k, i_x])
                        p_old_c = p_c

                ## rows
                valor_L_old, valor_A_old = L_f_v1v2_old[i_chan,i_x], A_f[i_k,p_old,p_old_c]   
                valor_L_new, valor_A_new = L_f_v1v2[i_chan,i_x], A_f[i_k,p_f,p_c]
                ## columns
                valor_L_old_c, valor_A_old_c = 0,0
                valor_L_new_c, valor_A_new_c = 0,0
                column = 0        
                if (i_x not in changes):
                    column = 1
                    valor_L_old_c, valor_A_old_c = L_c_v1v2_old[i_chan,i_x],A_f[i_k,p_old_c, p_old]
                    valor_L_new_c, valor_A_new_c = L_c_v1v2[i_chan,i_x], A_f[i_k,p_c,p_f]

                ovlp_1_old = ovlp_1_old + valor_L_old*valor_A_old + valor_L_old_c*valor_A_old_c*column
                ovlp_0_old = ovlp_0_old + (1-valor_L_old)*(1-valor_A_old) + (1-valor_L_old_c)*(1-valor_A_old_c)*column
                
                ovlp_1_new = ovlp_1_new + valor_L_new*valor_A_new + valor_L_new_c*valor_A_new_c*column
                ovlp_0_new = ovlp_0_new + (1-valor_L_new)*(1-valor_A_new) + (1-valor_L_new_c)*(1-valor_A_new_c)*column
                
    ovlp_1_def = ovlp_1_new - ovlp_1_old
    ovlp_0_def = ovlp_0_new- ovlp_0_old
   
    return ovlp_0_def, ovlp_1_def
### Some functions that numba do not have
@njit
def concatenate_numba_sinrep(a,b):
    size_a, size_b = len(a), len(b)
    no_double = []
    for i_b in range(size_b):
        if b[i_b] in a:
            size_b = size_b-1
        else:
            no_double.append(b[i_b])
    size = size_a + size_b
    c = np.zeros((size))
    c[0:size_a] = a
    c[size_a:] = no_double
    
    return a

@njit
def concatenate_numba(a,b):
    size_a, size_b = len(a), len(b)
    size = size_a + size_b
    c = np.zeros((size))
    c[0:size_a] = a
    c[size_a:] = b 
    
    return c

@njit
def sum_numba(S):
    Nx = S.shape[0]
    Ny = S.shape[1]
    
    suma = 0
    for i_s in range(Nx):
        suma = suma + sum(S[i_s,:])
        
    return suma
@njit
def sum_numba_filas(S): # 2 dimensions
    Nx = S.shape[0]
    Ny = S.shape[1]
    suma = np.zeros((Nx))
    for i_s in range(Nx):
        suma_c = 0 # sum of the column
        for i_y in range(Ny):
            suma[i_s] = suma_c + S[i_y,i_s] 
        
    return suma

@jit(nopython = True)
def hamiltonian_prob(Edges_NoL, Edges_L, overlap_0, overlap_1, alpha, beta):

    A_1 = overlap_1 + alpha
    B_1 = (Edges_L - overlap_1 + beta)
    C_1 = Edges_L + alpha + beta
    
    A_0 = overlap_0 + alpha
    B_0 = (Edges_NoL - overlap_0 + beta)
    C_0 = Edges_NoL + alpha + beta
    
    #  [ math.lgamma(n+1) == log(n!) ]
    H1 = math.lgamma(A_1)+ math.lgamma(B_1) - math.lgamma(C_1) 
    H0 = math.lgamma(A_0)+ math.lgamma(B_0) - math.lgamma(C_0) 
    
    H = -(H1 + H0)
    return H
    
    
@jit(nopython=True)
def overlap_total_prob(L_f, A_f, P_inv_f):
    Nx = L_f.shape[0]
    Ny = L_f.shape[1]
    K = A_f.shape[0]
    
    ovlp_0 = np.zeros((K))
    ovlp_1 = np.zeros((K))
    for k in range(0,K):
        for f in range(0,Ny): 
            for c in range(0,Nx):
                p_f=int(P_inv_f[k,f])
                p_c=int(P_inv_f[k,c])  
                
                valor_L, valor_A = L_f[f,c], A_f[k,p_f,p_c]
                
                ovlp_0[k] = ovlp_0[k] + (1-valor_L)*(1-valor_A )
                ovlp_1[k] = ovlp_1[k] + valor_L*valor_A
                
                
    ovlp_1 = int(sum(ovlp_1))
    ovlp_0 = int(sum(ovlp_0))
    return ovlp_0, ovlp_1


@jit(nopython=True) # The blueprint is the average of the observations (taking into account the mapping)  
def L_wiring(A_f, P_inv_f):
    
    Nx = A_f.shape[1]
    Ny = A_f.shape[2]
    K = A_f.shape[0]
    L_new_f = np.zeros((Nx,Ny))
    
    for i in range(0,Nx):
        for j in range(0,Ny):
            for k in range(0,K):
        
                p1 = int(P_inv_f[k,i]) # Mapping of the observations
                p2 = int(P_inv_f[k,j]) # Mapping of the observations
                L_new_f[i,j] += A_f[k,p1,p2]
            valor_lnew=1/K* L_new_f[i,j]
            L_new_f[i,j] = round( valor_lnew ) 
            # If valor_lnew = 0, L=0 (we could establish L=1, but it is more probable to not have a connection)
    
    return L_new_f
@njit
def custom_round(number):
    if number >= 0.5:
        return 1
    else:
        return 0

@njit
def random_menos_mas(number):
    if number >= 0.5:
        return 1
    else:
        return -1
def ground_state(A,K,Nx, alpha, beta):
    P_new, P_inv_new = np.zeros((K,Nx)),np.zeros((K,Nx))
    for k in range(0,K):
        for i in range(0,Nx):
            P_new[k,i]=i 
            P_inv_new[k,i]=i            
    L_new = np.zeros((Nx,Nx))
    L_new =  L_wiring(A, P_inv_new)
    
    Edges_sum = K*Nx*Nx
    Edges_L = K*(sum(sum(L_new))) # Los edges=1 que tiene la matriz A
    Edges_NoL = Edges_sum - Edges_L #Los edges=0 que tiene la matriz A
    overlap_0, overlap_1 = overlap_total_prob(L_new,A,P_inv_new)
    
    Energy_new = hamiltonian_prob(Edges_NoL, Edges_L, overlap_0, overlap_1, alpha, beta)

    return P_new, P_inv_new, L_new, Energy_new   

@jit(nopython=True)
def L_wiring_change_filas(m1,L_old,A_f, i_change, j_change, P_inv_old):
    
    Nx = A_f.shape[1]
    K = A_f.shape[0] 
    
    L_new_f = L_old.copy()
    L_f_v1v2,L_c_v1v2  = np.zeros((2,Nx)),np.zeros((2,Nx))
    
    ## changes
    changes = np.array([i_change, j_change])
    changes_new = np.array([j_change, i_change])
    
    ## Old values
    suma_old =(np.sum(L_old[i_change,:]) + np.sum(L_old[j_change,:]) + np.sum(L_old[:,i_change]) + np.sum(L_old[:,j_change]))-L_old[i_change,j_change]-L_old[j_change,j_change]-L_old[i_change,i_change]-L_old[j_change,i_change]
    
    ## New row and column
    L_new_f[i_change,:], L_new_f[j_change,:], L_new_f[:,i_change], L_new_f[:,j_change] = np.zeros(Nx),np.zeros(Nx),np.zeros(Nx),np.zeros(Nx)
    

    suma_1 = 0
    
    for i_i,i_chan in enumerate(changes): #Only looking at the 
        ############### ROWS & COLUMNS
        for i_x in range(Nx):
            if (i_x in changes): # The rest of nodes
                for k in range(K):
                    if (k == m1):
                        chan_new = changes_new[i_i]
                        
                        if (i_x == changes_new[0]):
                            p2_2 = int(P_inv_old[k,changes_new[1]])
                        else:
                            p2_2 = int(P_inv_old[k,changes_new[0]])
                    else:
                        chan_new = changes[i_i]
                        p2_2 = int(P_inv_old[k,i_x])
                    p1_2 = int (P_inv_old[k,chan_new])

                    L_f_v1v2[i_i,i_x] += A_f[k,p1_2,p2_2]
                    L_c_v1v2[i_i,i_x] += A_f[k,p2_2,p1_2]

            else: # The nodes that change
                for k in range(K):
                    if (k == m1):
                        chan_new = changes_new[i_i]
                    else:
                        chan_new = changes[i_i]
                    
                    
                    p1_2 = int (P_inv_old[k,chan_new])
                    p2_2 = int(P_inv_old[k,i_x])
                    
                    L_f_v1v2[i_i,i_x] += A_f[k,p1_2,p2_2]
                    L_c_v1v2[i_i,i_x] += A_f[k,p2_2,p1_2]
                    
                    
                    
            valor_lnew_2=1/K*  L_f_v1v2[i_i,i_x]
            L_f_v1v2[i_i,i_x] = round( valor_lnew_2 )

            valor_lnew_c=1/K* L_c_v1v2[i_i,i_x]
            L_c_v1v2[i_i,i_x] = round( valor_lnew_c)
                    
    suma_new = np.sum(L_f_v1v2[0,:]) + np.sum(L_f_v1v2[1,:]) + np.sum(L_c_v1v2[0,:]) + np.sum(L_c_v1v2[1,:])-L_c_v1v2[0,j_change]-L_c_v1v2[0,i_change]-L_c_v1v2[1,j_change]-L_c_v1v2[1,i_change]
    suma_1 = suma_new - suma_old
    return L_f_v1v2,L_c_v1v2, suma_1    

@jit(nopython=True)
def L_wiring_overlap_change_filas(m1,L_f_v1v2_old,L_c_v1v2_old,A_f, i_change, j_change, P_inv_old, pp1, pp2):
    
    Nx = A_f.shape[1]
    K = A_f.shape[0] 

    #### Para la parte del WIRING    
    L_f_v1v2,L_c_v1v2  = np.zeros((2,Nx)),np.zeros((2,Nx))
    
    ## changes
    changes = np.array([i_change, j_change])
    changes_new = np.array([j_change, i_change])
    
    ## Old values
    suma_old =(np.sum(L_f_v1v2_old[0,:]) + np.sum(L_f_v1v2_old[1,:]) + np.sum(L_c_v1v2_old[0,:]) + np.sum(L_c_v1v2_old[1,:]))-L_f_v1v2_old[0,j_change]-L_f_v1v2_old[1,j_change]-L_f_v1v2_old[0,i_change]-L_f_v1v2_old[1,i_change]
    
    ## New row and column
    ##### Para la parte del OVERLAP
    ovlp_0_new , ovlp_1_new = 0,0
    ovlp_0_old , ovlp_1_old = 0,0
    ovlp_0_dif, ovlp_1_dif = 0,0
    changes_p = np.array([pp1, pp2], dtype = np.int32)  
    changes_p_old = np.array([pp2, pp1], dtype = np.int32)
    suma_1 = 0
    
    for i_i,i_chan in enumerate(changes): #Only looking at the 
        ############### ROWS & COLUMNS
        for i_x in range(Nx):
            if (i_x in changes): # The rest of nodes
                for k in range(K):
                    if (k == m1):
                        chan_new = changes_new[i_i]
                        
                        if (i_x == changes_new[0]):
                            p2_2 = int(P_inv_old[k,changes_new[1]])
                        else:
                            p2_2 = int(P_inv_old[k,changes_new[0]])
                    else:
                        chan_new = changes[i_i]
                        p2_2 = int(P_inv_old[k,i_x])
                    p1_2 = int (P_inv_old[k,chan_new])

                    L_f_v1v2[i_i,i_x] += A_f[k,p1_2,p2_2]
                    L_c_v1v2[i_i,i_x] += A_f[k,p2_2,p1_2]

            else: # The nodes that change
                for k in range(K):
                    if (k == m1):
                        chan_new = changes_new[i_i]
                    else:
                        chan_new = changes[i_i]
                    
                    
                    p1_2 = int (P_inv_old[k,chan_new])
                    p2_2 = int(P_inv_old[k,i_x])
                    
                    L_f_v1v2[i_i,i_x] += A_f[k,p1_2,p2_2]
                    L_c_v1v2[i_i,i_x] += A_f[k,p2_2,p1_2]
                    
                    
                    
            valor_lnew_2=1/K*  L_f_v1v2[i_i,i_x]
            L_f_v1v2[i_i,i_x] = round( valor_lnew_2 )

            valor_lnew_c=1/K* L_c_v1v2[i_i,i_x]
            L_c_v1v2[i_i,i_x] = round( valor_lnew_c)
                    
            ## Una vez que tenemos los valores de la L podemos mirar el overlap!
            for i_k in range(K):
                if i_k != m1: # Only changes in m1
                    p_f = int(P_inv_old[i_k,i_chan])
                    p_c = int(P_inv_old[i_k,i_x])
                    p_old, p_old_c = p_f, p_c
                else:
                    p_f = changes_p[i_i]
                    p_old = changes_p_old[i_i]
                    if (i_x == i_change):
                        p_c = pp1
                        p_old_c = pp2
                    elif(i_x == j_change):
                        p_c = pp2
                        p_old_c = pp1
                    else: # Only changes in the nodes i,j
                        p_c = int(P_inv_old[i_k, i_x])
                        p_old_c = p_c
            
                valor_L_old, valor_A_old = L_f_v1v2_old[i_i,i_x], A_f[i_k,p_old,p_old_c]   
                valor_L_new, valor_A_new = L_f_v1v2[i_i,i_x], A_f[i_k,p_f,p_c]
                ## columns
                valor_L_old_c, valor_A_old_c = 0,0
                valor_L_new_c, valor_A_new_c = 0,0
                column = 0        
                if (i_x not in changes):
                    column = 1
                    valor_L_old_c, valor_A_old_c = L_c_v1v2_old[i_i,i_x],A_f[i_k,p_old_c, p_old]
                    valor_L_new_c, valor_A_new_c = L_c_v1v2[i_i,i_x], A_f[i_k,p_c,p_f]
    
                ovlp_1_old = ovlp_1_old + valor_L_old*valor_A_old + valor_L_old_c*valor_A_old_c*column
                ovlp_0_old = ovlp_0_old + (1-valor_L_old)*(1-valor_A_old) + (1-valor_L_old_c)*(1-valor_A_old_c)*column
                
                ovlp_1_new = ovlp_1_new + valor_L_new*valor_A_new + valor_L_new_c*valor_A_new_c*column
                ovlp_0_new = ovlp_0_new + (1-valor_L_new)*(1-valor_A_new) + (1-valor_L_new_c)*(1-valor_A_new_c)*column
                

    
                    
    suma_new = np.sum(L_f_v1v2[0,:]) + np.sum(L_f_v1v2[1,:]) + np.sum(L_c_v1v2[0,:]) + np.sum(L_c_v1v2[1,:])-L_c_v1v2[0,j_change]-L_c_v1v2[0,i_change]-L_c_v1v2[1,j_change]-L_c_v1v2[1,i_change]
    suma_1 = suma_new - suma_old

    ## Parte del overlap 
    ovlp_1_def = ovlp_1_new - ovlp_1_old
    ovlp_0_def = ovlp_0_new- ovlp_0_old



    
    return L_f_v1v2,L_c_v1v2, suma_1, ovlp_0_def, ovlp_1_def

@jit(nopython = True)
def parallel_L_knwn_change_groups(c_parallel,alpha,beta,fijado_f,groups_f, N_groups_f, start_f, end_f,
                       A, P_t_f, P_inv_t_f, L_t_f,suma_L, Energy_t_f, ovlp_t_f0, ovlp_t_f1, tempers_f,tempers_odd, tempers_even,
                                    c_accep, c_odd, m_old, round_trip_old, c_rounds):
    
    # fijado_f : To let one of the networks fixed or not (If only two networks it doesn't matter)
    # c_parallel: counter to change temperatures
    # start_f, end,f
    # N_groups_f: Number of groups
    # groups_f: Directly the ggroup numbeer from the position
    ### Movement:
    # 1) Choose one node and see their group (also we can choose a group first)
    # 2) Choose another random node inside this group 
    # 3) Each 4 movements, we propose change the temperatures

    N_t = L_t_f.shape[0] # Numbers of tempers
    Nx = L_t_f.shape[1] # Numbers of nodes
    K = A.shape[0] # Numbers of networks
    Edges_sum = K*Nx*Nx
   
    if (c_parallel <4*Nx): # If not change of temperatures
        c_parallel += 1
        
        #Choose the networks to propose the change (in all temperatures)
        if (fijado_f == 1):
            m1 = np.random.randint(K-1)+1 #One networks
        else:
            m1 = np.random.randint(K)    
        
        for k_nt in range(N_t):    
            # 1) Choose a random node (node in A)
            
            # Si cojo el mapping vale siempre el anchor
            v1 = np.random.randint(start_f[0], Nx)
            grupo = groups_f[k_nt,m1,v1]
            start, end = start_f[grupo], end_f[grupo]
            size_group = end - start
            todos = size_group 
            if (size_group > 1): #No grupos de uno
        
                v2 = np.random.randint(start, end)
                while (v2 == v1):
                    v2 = np.random.randint(start, end)

                v1_mapping = P_inv_t_f[k_nt, m1, v1]
                v2_mapping = P_inv_t_f[k_nt, m1, v2]
    
               ### Only changes in the specific columns and rows
                L_f_v1v2, L_f_v1v2_old = np.zeros((2,Nx)),np.zeros((2,Nx))
                L_c_v1v2, L_c_v1v2_old = np.zeros((2,Nx)),np.zeros((2,Nx))
    
                pp_1 = int(P_inv_t_f[k_nt,m1,v2]) # v2_mapping
                pp_2 = int(P_inv_t_f[k_nt,m1,v1]) # v1_mapping
    
                L_f_v1v2_old[0,:],L_f_v1v2_old[1,:] = L_t_f[k_nt,v1,:].copy(),L_t_f[k_nt,v2,:].copy()
                L_c_v1v2_old[0,:],L_c_v1v2_old[1,:] = L_t_f[k_nt,:,v1].copy(),L_t_f[k_nt,:,v2].copy()
                
                # Changes in the blueprint
                L_f_v1v2,L_c_v1v2 , suma_aux,ovl_0_new, ovl_1_new = L_wiring_overlap_change_filas(m1,L_f_v1v2_old,L_c_v1v2_old, A, v1, v2, P_inv_t_f[k_nt,:,:], pp_1, pp_2)
                
                # Parameters to change
                Edges_L = K*(suma_L[k_nt] + suma_aux)
                Edges_NoL = Edges_sum - Edges_L
                overlap_0,overlap_1 = ovlp_t_f0[k_nt] + ovl_0_new, ovlp_t_f1[k_nt] + ovl_1_new
                Energy_bucle = hamiltonian_prob(Edges_NoL, Edges_L, overlap_0,overlap_1,alpha, beta)
                dE_sampler = Energy_bucle - Energy_t_f[k_nt]
                dE_t = dE_sampler
                ## borrar
                # if k_nt == 1:
                #     print(dE_t)
                ## borrar
                
                ### UPDATE
                if (dE_t < 0):
                    
                    P_aux1 = P_t_f[k_nt, m1,pp_2]
                    P_t_f[k_nt, m1, pp_2] = P_t_f[k_nt, m1,pp_1]
                    P_t_f[k_nt, m1, pp_1] = P_aux1
    
                    P_invaux1 = P_inv_t_f[k_nt,m1,v2]
                    P_inv_t_f[k_nt,m1,v2] = P_inv_t_f[k_nt,m1,v1]
                    P_inv_t_f[k_nt,m1,v1] = P_invaux1
    
                    Energy_t_f[k_nt] = Energy_t_f[k_nt] + dE_t
                    ovlp_t_f0[k_nt] = overlap_0
                    ovlp_t_f1[k_nt] = overlap_1
                    L_t_f[k_nt,v1, :],L_t_f[k_nt,v2, :] = L_f_v1v2[0,:],L_f_v1v2[1,:]
                    L_t_f[k_nt,:,v1],L_t_f[k_nt,:,v2] = L_c_v1v2[0,:],L_c_v1v2[1,:]
                    suma_L[k_nt] = suma_L[k_nt] + suma_aux
    
                elif (np.random.rand() < np.exp(-dE_t*tempers_f[k_nt])):
    
                    P_aux1 = P_t_f[k_nt, m1,pp_2]
                    P_t_f[k_nt, m1, pp_2] = P_t_f[k_nt, m1,pp_1]
                    P_t_f[k_nt, m1, pp_1] = P_aux1
    
                    P_invaux1 = P_inv_t_f[k_nt,m1,v2]
                    P_inv_t_f[k_nt,m1,v2] = P_inv_t_f[k_nt,m1,v1]
                    P_inv_t_f[k_nt,m1,v1] = P_invaux1
    
                    Energy_t_f[k_nt] = Energy_t_f[k_nt] + dE_t
                    ovlp_t_f0[k_nt] = overlap_0
                    ovlp_t_f1[k_nt] = overlap_1
                    L_t_f[k_nt,v1, :],L_t_f[k_nt,v2, :] = L_f_v1v2[0,:].copy(),L_f_v1v2[1,:].copy()
                    L_t_f[k_nt,:,v1],L_t_f[k_nt,:,v2] = L_c_v1v2[0,:].copy(),L_c_v1v2[1,:].copy()
                    suma_L[k_nt] = suma_L[k_nt] + suma_aux

    else: # Change in temperatures
        c_parallel = 0
        mt1 = np.random.randint(N_t) 
        if mt1 == 0:
            mt2 = 1
        elif mt1 == (N_t - 1):
            mt2 = mt1 -1
        else:
            mt2 = mt1 + random_menos_mas(np.random.rand())
        


        #Probar el cambio de energía
        #Cambio del 1:
        Energy_1 = Energy_t_f[mt1]
        #Cambio del 2:
        Energy_2 = Energy_t_f[mt2] 

        dE_parallel = -(tempers_f[mt1]-tempers_f[mt2])*(Energy_1-Energy_2)
        if (dE_parallel <0):#Se acepta directamente
            #Auxliar
            P_aux1 = (P_t_f[mt1, :, :]).copy()
            P_inv_aux1 = (P_inv_t_f[mt1,:,:]).copy()
            L_aux1 = (L_t_f[mt1,:,:]).copy()

            ovlp_aux0 = (ovlp_t_f0[mt1])
            ovlp_aux1 = (ovlp_t_f1[mt1])
            suma_aux = suma_L[mt1]
            #El 1 lo guardo en el 2
            Energy_t_f[mt1] = Energy_2
            P_t_f[mt1,:,:] = (P_t_f[mt2, :, :]).copy()
            P_inv_t_f[mt1,:,:] = (P_inv_t_f[mt2,:,:]).copy()
            L_t_f[mt1,:,:] = (L_t_f[mt2,:,:]).copy()
            suma_L[mt1] = suma_L[mt2]
            ovlp_t_f0[mt1] = ovlp_t_f0[mt2]
            ovlp_t_f1[mt1] = ovlp_t_f1[mt2]

            #El 2 lo guardo en el 1 (que previamente lo he guardado bien)
            Energy_t_f[mt2] = Energy_1
            P_t_f[mt2,:,:] = P_aux1.copy()
            P_inv_t_f[mt2,:,:] = P_inv_aux1.copy()
            L_t_f[mt2,:,:] = L_aux1.copy()

            ovlp_t_f0[mt2] = ovlp_aux0
            ovlp_t_f1[mt2] = ovlp_aux1
            suma_L[mt2] = suma_aux
        elif (np.random.rand() < np.exp(-dE_parallel)):
            P_aux1 = (P_t_f[mt1, :, :]).copy()
            P_inv_aux1 = (P_inv_t_f[mt1,:,:]).copy()
            L_aux1 = (L_t_f[mt1,:,:]).copy()

            ovlp_aux0 = (ovlp_t_f0[mt1])
            ovlp_aux1 = (ovlp_t_f1[mt1])
            suma_aux = suma_L[mt1]
            #El 1 lo guardo en el 2
            Energy_t_f[mt1] = Energy_2
            P_t_f[mt1,:,:] = (P_t_f[mt2, :, :]).copy()
            P_inv_t_f[mt1,:,:] = (P_inv_t_f[mt2,:,:]).copy()
            L_t_f[mt1,:,:] = (L_t_f[mt2,:,:]).copy()
            suma_L[mt1] = suma_L[mt2]
            ovlp_t_f0[mt1] = ovlp_t_f0[mt2]
            ovlp_t_f1[mt1] = ovlp_t_f1[mt2]

            #El 2 lo guardo en el 1 (que previamente lo he guardado bien)
            Energy_t_f[mt2] = Energy_1
            P_t_f[mt2,:,:] = P_aux1.copy()
            P_inv_t_f[mt2,:,:] = P_inv_aux1.copy()
            L_t_f[mt2,:,:] = L_aux1.copy()

            ovlp_t_f0[mt2] = ovlp_aux0
            ovlp_t_f1[mt2] = ovlp_aux1
            suma_L[mt2] = suma_aux

        # c_parallel = 0
        # if c_odd == 1:
        #     c_odd = 0
        #     mt1_tempers = tempers_odd[0,:]
        #     mt2_tempers = tempers_odd[1,:]

        # else:
        #     c_odd = 1
        #     mt1_tempers = tempers_even[0,:]
        #     mt2_tempers = tempers_even[1,:]

            
        # for mt1, mt2  in zip(mt1_tempers, mt2_tempers):
        
        #     #Probar el cambio de energía
        #     #Cambio del 1:
        #     Energy_1 = Energy_t_f[mt1]
        #     #Cambio del 2:
        #     Energy_2 = Energy_t_f[mt2] 

    
        #     dE_parallel = -(tempers_f[mt1]-tempers_f[mt2])*(Energy_1-Energy_2)
                
        #     if (dE_parallel <0):
        #         P_aux1 = (P_t_f[mt1, :, :]).copy()
        #         P_inv_aux1 = (P_inv_t_f[mt1,:,:]).copy()
        #         L_aux1 = (L_t_f[mt1,:,:]).copy()

        #         ovlp_aux0 = (ovlp_t_f0[mt1])
        #         ovlp_aux1 = (ovlp_t_f1[mt1])
        #         suma_aux = suma_L[mt1]

        #         # 1 --> 2
        #         Energy_t_f[mt1] = Energy_2
        #         P_t_f[mt1,:,:] = (P_t_f[mt2, :, :]).copy()
        #         P_inv_t_f[mt1,:,:] = (P_inv_t_f[mt2,:,:]).copy()
        #         L_t_f[mt1,:,:] = (L_t_f[mt2,:,:]).copy()

        #         ovlp_t_f0[mt1] = ovlp_t_f0[mt2]
        #         ovlp_t_f1[mt1] = ovlp_t_f1[mt2]
        #         suma_L[mt1] = suma_L[mt2]

        #         # 2 --> 1
        #         Energy_t_f[mt2] = Energy_1
        #         P_t_f[mt2,:,:] = P_aux1
        #         P_inv_t_f[mt2,:,:] = P_inv_aux1
        #         L_t_f[mt2,:,:] = L_aux1

        #         ovlp_t_f0[mt2] = ovlp_aux0
        #         ovlp_t_f1[mt2] = ovlp_aux1
        #         suma_L[mt2] = suma_aux
                
        #         if m_old == mt1:
        #             m_old = mt2
        #         elif m_old == mt2:
        #             m_old = mt1
        #         if m_old == 0 and round_trip_old == -1:
        #             c_rounds += 1
        #             round_trip_old = 1
        #         elif m_old == (N_t -1) and round_trip_old == 1:
        #             c_rounds += 1
        #             rount_trip_old = -1
                    
                
                
        #         if tempers_f[mt1] == 1 or tempers_f[mt2] == 1:
        #             c_accep += 1

        #     elif (np.random.rand() < np.exp(-dE_parallel)):
        #         P_aux1 = (P_t_f[mt1, :, :]).copy()
        #         P_inv_aux1 = (P_inv_t_f[mt1,:,:]).copy()
        #         L_aux1 = (L_t_f[mt1,:,:]).copy()

        #         ovlp_aux0 = (ovlp_t_f0[mt1])
        #         ovlp_aux1 = (ovlp_t_f1[mt1])
        #         suma_aux = suma_L[mt1]

        #         # 1 --> 2
        #         Energy_t_f[mt1] = Energy_2
        #         P_t_f[mt1,:,:] = (P_t_f[mt2, :, :]).copy()
        #         P_inv_t_f[mt1,:,:] = (P_inv_t_f[mt2,:,:]).copy()
        #         L_t_f[mt1,:,:] = (L_t_f[mt2,:,:]).copy()

        #         ovlp_t_f0[mt1] = ovlp_t_f0[mt2]
        #         ovlp_t_f1[mt1] = ovlp_t_f1[mt2]
        #         suma_L[mt1] = suma_L[mt2]

        #         # 2 --> 1
        #         Energy_t_f[mt2] = Energy_1
        #         P_t_f[mt2,:,:] = P_aux1
        #         P_inv_t_f[mt2,:,:] = P_inv_aux1
        #         L_t_f[mt2,:,:] = L_aux1

        #         ovlp_t_f0[mt2] = ovlp_aux0
        #         ovlp_t_f1[mt2] = ovlp_aux1
        #         suma_L[mt2] = suma_aux
                
        #         if m_old == mt1:
        #             m_old = mt2
        #         elif m_old == mt2:
        #             m_old = mt1
        #         if m_old == 0 and round_trip_old == -1:
        #             c_rounds += 1
        #             round_trip_old = 1
        #         elif m_old == (N_t -1) and round_trip_old == 1:
        #             c_rounds += 1
        #             round_trip_old = -1
                    
                
        #         if tempers_f[mt1] == 1 or tempers_f[mt2] == 1:
        #             c_accep += 1

  
    return c_parallel, P_t_f, P_inv_t_f,Energy_t_f, ovlp_t_f0, ovlp_t_f1, L_t_f, suma_L, c_accep, c_odd, m_old, round_trip_old, c_rounds

# Iteraciones MC

In [15]:
parejas = 0 # Para saber si coger parejas o no
fijado = 0 # Para saber si fijar la primera red o no

In [16]:
P_new, P_inv_new = np.zeros((K,Nx)),np.zeros((K,Nx))
for k in range(0,K):
    for i in range(0,Nx):
        P_new[k,i]=i 
        P_inv_new[k,i]=i            
L_new = np.zeros((Nx,Ny)) #La propuesta de cambio
L_new =  L_wiring(A, P_inv_new)

Edges_sum = K*Nx*Nx
Edges_L = K*(sum(sum(L_new))) # Los edges=1 que tiene la matriz A
Edges_NoL = Edges_sum - Edges_L #Los edges=0 que tiene la matriz A
overlap_0, overlap_1 = overlap_total_prob(L_new,A,P_inv_new)     
Energy_new = hamiltonian_prob(Edges_NoL, Edges_L, overlap_0, overlap_1, alpha, beta)
Energy_ground_truth = Energy_new
print('Energía con permutación de media de matrices:', Energy_new)
print('L_new', np.sum(L_new), np.sum(A))
print( 'overlap_0:', overlap_0,'/overlap_1:', overlap_1)
print('%Overlaps:',100*(overlap_0+overlap_1)/Edges_sum ,'%')

# Nuestro problema
ovlp_ini0, ovlp_ini1 = overlap_total_prob(L_ini,A,P_inv_ini_0)

# METIENDO LAS TEMPERATURAS
ovlp_ini0_t = np.zeros((N_t)) + ovlp_ini0
ovlp_ini1_t = np.zeros((N_t)) + ovlp_ini1
ovlp_ini0_t, ovlp_ini1_t = ovlp_ini0_t.astype(int), ovlp_ini1_t.astype(int)

P_t = np.zeros((N_t,K,Nx))
P_inv_t = np.zeros((N_t,K,Nx))
L_t = np.zeros((N_t,Nx, Ny))
Energy_t, Energy_ini_t = np.zeros((N_t)), np.zeros((N_t))
Edges_L_t, Edges_NoL_t = np.zeros((N_t)), np.zeros((N_t))
groups_t = np.zeros((N_t, K, Nx))


# Inicializamos
ovlp_t0, ovlp_t1 = np.zeros((N_t)) + ovlp_ini0, np.zeros((N_t)) + ovlp_ini1
P_t = (P_ini_t).copy()
P_inv_t = (P_inv_ini_t).copy()
L_t = (L_ini_t).copy()
groups_t = groups_ini_t.copy()
#Convertirlos en enteros
ovlp_t0, ovlp_t1 = ovlp_t0.astype(int), ovlp_t1.astype(int)
P_t = P_t.astype(int)
P_inv_t = P_inv_t.astype(int)
L_t = L_t.astype(int)
groups_t = groups_t.astype(int)

## Vamos a guardar las aceptaciones
Edges_L = K*sum(sum(L_ini))
Edges_NoL = Edges_sum - Edges_L            
Energy_ini = hamiltonian_prob(Edges_NoL, Edges_L, ovlp_ini0, ovlp_ini1 ,alpha, beta)
Energy_ini_t = np.zeros((N_t))+ Energy_ini 
print()

print('Energía inicial:', Energy_ini)
print( 'overlap_0:', ovlp_ini0,'/overlap_1:', ovlp_ini1,'//Edges', Edges_L, Edges_NoL)
print(np.sum(L_ini), np.sum(A), np.sum(P_inv_ini_0-np.arange(Nx)))
print()
E_gs = Energy_new


Energía con permutación de media de matrices: 144388.1567874623
L_new 14262.0 53438.0
overlap_0: 2997012 /overlap_1: 28524
%Overlaps: 99.18326804241997 %

Energía inicial: 263821.1549609881
overlap_0: 2947326 /overlap_1: 5432 //Edges 55118.0 2995332.0
27559.0 53438.0 0



In [17]:
P_new, P_inv_new = np.zeros((K,Nx)),np.zeros((K,Nx))
for k in range(0,K):
    for i in range(0,Nx):
        P_new[k,i]=i 
        P_inv_new[k,i]=i            
L_new = np.zeros((Nx,Ny)) #La propuesta de cambio
L_new =  L_wiring(A, P_inv_new)

Edges_sum = K*Nx*Nx
Edges_L = K*(sum(sum(L_new))) # Los edges=1 que tiene la matriz A
Edges_NoL = Edges_sum - Edges_L #Los edges=0 que tiene la matriz A
overlap_0, overlap_1 = overlap_total_prob(L_new,A,P_inv_new)     
Energy_new = hamiltonian_prob(Edges_NoL, Edges_L, overlap_0, overlap_1, alpha, beta)
Energy_ground_truth = Energy_new
print('Energía con permutación de media de matrices:', Energy_new)
print( 'overlap_0:', overlap_0,'/overlap_1:', overlap_1)
print('%Overlaps:',100*(overlap_0+overlap_1)/Edges_sum ,'%')

# Nuestro problema
ovlp_ini0, ovlp_ini1 = overlap_total_prob(L_ini,A,P_inv_ini_0)

# METIENDO LAS TEMPERATURAS
ovlp_ini0_t = np.zeros((N_t)) + ovlp_ini0
ovlp_ini1_t = np.zeros((N_t)) + ovlp_ini1
ovlp_ini0_t, ovlp_ini1_t = ovlp_ini0_t.astype(int), ovlp_ini1_t.astype(int)

P_t = np.zeros((N_t,K,Nx))
P_inv_t = np.zeros((N_t,K,Nx))
L_t = np.zeros((N_t,Nx, Ny))
Energy_t, Energy_ini_t = np.zeros((N_t)), np.zeros((N_t))
Edges_L_t, Edges_NoL_t = np.zeros((N_t)), np.zeros((N_t))
groups_t = np.zeros((N_t, K, Nx))
groups_anch_t = np.zeros((N_t, K, Nx))

# Inicializamos
ovlp_t0, ovlp_t1 = np.zeros((N_t)) + ovlp_ini0, np.zeros((N_t)) + ovlp_ini1
P_t = (P_ini_t).copy()
P_inv_t = (P_inv_ini_t).copy()
L_t = (L_ini_t).copy()
groups_t = groups_ini_t.copy()
groups_anch_t = groups_anch_ini_t.copy()
#Convertirlos en enteros
ovlp_t0, ovlp_t1 = ovlp_t0.astype(int), ovlp_t1.astype(int)
P_t = P_t.astype(int)
P_inv_t = P_inv_t.astype(int)
L_t = L_t.astype(int)
groups_t = groups_t.astype(int)
groups_anch_t = groups_anch_t.astype(int)
## Vamos a guardar las aceptaciones
Edges_L = K*sum(sum(L_ini))
Edges_NoL = Edges_sum - Edges_L            
Energy_ini = hamiltonian_prob(Edges_NoL, Edges_L, ovlp_ini0, ovlp_ini1 ,alpha, beta)
Energy_ini_t = np.zeros((N_t))+ Energy_ini 
print()

print('Energía inicial:', Energy_ini)
print()

#######################################
start = time.time()
count_mc = 0
#### Bloque del wiring
for k_nt in range(N_t):
    L_t[k_nt,:,:] =  L_wiring(A, P_inv_t[k_nt,:,:])
    Edges_L_t[k_nt] = K*sum(sum(L_t[k_nt,:,:]))
    Edges_NoL_t[k_nt] = Edges_sum - Edges_L_t[k_nt]
    overlap_0, overlap_1 = overlap_total_prob(L_t[k_nt,:,:],A,P_inv_t[k_nt,:,:])
    Energy_t[k_nt] = hamiltonian_prob(Edges_NoL_t[k_nt], Edges_L_t[k_nt], overlap_0, overlap_1,alpha, beta)
    ovlp_t0[k_nt], ovlp_t1[k_nt] = overlap_0, overlap_1
#####

print('Energía INICIAL TRAS WIRING:', Energy_t[0]) 
print('overlaps 0:', ovlp_t0[0], '1:', ovlp_t1[0])
print('%Overlaps:',100*(ovlp_t0[0] + ovlp_t1[0])/Edges_sum ,'%')

print()
Energy_ini_wiring = Energy_t.copy()
############################################ MONTE CARLO
#Guardamos la energía
runs = 6
epochs = runs
PasosMC = 1000
energies_pasos = np.zeros((runs,N_t,PasosMC))
energies_pasos[:,:,0] = Energy_ini_wiring
count_parallel, count_unl = 0, 0

# Guardamos configuraciones
Long_corr = 50
Pasos_corr = 200

Pasos_store = int((PasosMC - (Pasos_corr-Long_corr))/Long_corr)
if (Pasos_store<0):
    Pasos_store = 0

print('Pasos_store', Pasos_store, '//runs:', Pasos_store*runs)
count_long, i_long = 0, 0 
P_store = np.zeros((runs,Pasos_store,N_t, K, Nx))
Energy_store = np.zeros((runs,Pasos_store, N_t))
print('Pasos_store_per_run',Pasos_store)
i_count_micro,i_count_micro2 = 0,0

P_intermediate_shot = np.zeros((2,N_t,K,Nx))
E_intermediate_shot = np.zeros((2,N_t))
steps_middle_0 = 50
steps_middle_1 = 100

suma_L_t = np.zeros(N_t)
for k_nt in range(N_t):
    suma_L_t[k_nt] = sum_numba(L_t[k_nt,:,:])
########################################################
L_run, Edges_L_run, Edges_noL_run, ov0_run, ov1_run, E_run, suma_run, P_run, P_inv_run = L_t.copy(), Edges_L_t.copy(), Edges_NoL_t.copy(), ovlp_t0.copy(), ovlp_t1.copy(), Energy_t.copy(), suma_L_t.copy(),P_t.copy(), P_inv_t.copy()
Edges_L_run, Edges_noL_run, ov0_run, ov1_run = Edges_L_run.astype(int), Edges_noL_run.astype(int), ov0_run.astype(int), ov1_run.astype(int)

for i_epoch in range(runs):
    print('i_run', i_epoch)
    L_t, ovlp_t0, ovlp_t1, Energy_t,suma_L_t,P_t, P_inv_t = L_run.copy(), ov0_run.copy(), ov1_run.copy(), E_run.copy(), suma_run.copy(), P_run.copy(), P_inv_run.copy()
    i_long,count_long = 0,0
    c_accep, c_odd, c_rounds = 0,0,0
    m_old, round_trip_old = 0,1

    
    for i_mc in tqdm(range(0,PasosMC)):
    ### GUARDAR CONFIGURACIONES
        if (i_mc > (Pasos_corr-Long_corr)):
            count_long += 1
            if (count_long == Long_corr):
                count_long = 0
                P_store[i_epoch,i_long,:,:,:] = P_inv_t.copy()
                Energy_store[i_epoch,i_long,:] = Energy_t.copy()
                i_long += 1
                print('Step', i_mc, 'Energy (temper = 1):', Energy_t[4])
        ### FIN GUARDAR CONFIGURACIONES 
        
        if (i_mc == steps_middle_0 ):
            P_intermediate_shot[0,:,:,:] = P_inv_t.copy()
            E_intermediate_shot[0,:] = Energy_t.copy()
            
        if (i_mc == steps_middle_1 ):
            P_intermediate_shot[1,:,:,:] = P_inv_t.copy()
            E_intermediate_shot[1,:] = Energy_t.copy()
            
         
        energies_pasos[i_epoch,:, i_mc] = Energy_t[:]
            
        for i_micro in range(Nx):
            tempers_loop, tempers_odd_loop, tempers_even_loop =  tempers,tempers_odd, tempers_even
            tempers_odd_loop, tempers_even_loop =  tempers_odd_loop.astype(int), tempers_even_loop.astype(int)
            # print(tempers_loop.dtype, tempers_odd_loop.dtype, tempers_even_loop.dtype)
            count_parallel,P_t, P_inv_t, Energy_t, ovlp_t0, ovlp_t1, L_t,suma_L_t,c_accep, c_odd, m_old, round_trip_old, c_rounds = parallel_L_knwn_change_groups( count_parallel,alpha,beta,fijado,groups_anch_t,n_groups, start_anchor, 
                                                                                                             end_anchor,  A, P_t, P_inv_t, L_t,suma_L_t, Energy_t, ovlp_t0, 
                                                                                                             ovlp_t1,tempers_loop, tempers_odd_loop, tempers_even_loop,c_accep, c_odd, m_old, round_trip_old, c_rounds)
    

    print('Energía final paso:', Energy_t)
end = time.time()

print("Elapsed (after compilation) = %s" % (end - start))  

Energy_calculada = np.zeros((N_t))
for k_nt in range(N_t):
    Edges_L= K*sum(sum(L_t[k_nt,:,:]))
    Edges_NoL = Edges_sum - Edges_L
#     print('Para t=', tempers[k_nt], 'Edges_L', Edges_L)
    overlap_0, overlap_1 = overlap_total_prob(L_t[k_nt,:,:],A,P_inv_t[k_nt,:,:])
    Energy_calculada[k_nt] = hamiltonian_prob(Edges_NoL, Edges_L, overlap_0,overlap_1,alpha, beta)
    
print('Energía final:', Energy_t, 'Calculada:', Energy_calculada) 
print('overlaps_0:', ovlp_t0[0], 'overlap_1:', ovlp_t1[0], '%overlaps totales:', 100*(ovlp_t0[0] + ovlp_t1[0])/Edges_sum ,'%')


Energía con permutación de media de matrices: 144388.1567874623
overlap_0: 2997012 /overlap_1: 28524
%Overlaps: 99.18326804241997 %

Energía inicial: 263821.1549609881

Energía INICIAL TRAS WIRING: 226714.56570439774
overlaps 0: 2997012 1: 10232
%Overlaps: 98.58361881033946 %

Pasos_store 17 //runs: 102
Pasos_store_per_run 17
i_run 0


  0%|                                                  | 0/1000 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [18]:
groups_anch_t

array([[[-1, -1, -1, ..., 23, 23, 23],
        [-1, -1, -1, ..., 23, 23, 23]],

       [[-1, -1, -1, ..., 23, 23, 23],
        [-1, -1, -1, ..., 23, 23, 23]],

       [[-1, -1, -1, ..., 23, 23, 23],
        [-1, -1, -1, ..., 23, 23, 23]],

       ...,

       [[-1, -1, -1, ..., 23, 23, 23],
        [-1, -1, -1, ..., 23, 23, 23]],

       [[-1, -1, -1, ..., 23, 23, 23],
        [-1, -1, -1, ..., 23, 23, 23]],

       [[-1, -1, -1, ..., 23, 23, 23],
        [-1, -1, -1, ..., 23, 23, 23]]])

In [ ]:
d_s = {}
d_s["P_sto"] = P_store
d_s["E_sto"] = Energy_store
d_s["start_groups"] = start_groups
d_s["end_groups"] = end_groups
d_s["Datos_energy"] = energies_pasos
d_s["E_groundtruth"] = E_gs
d_s["Pasos_corr_ini"] = Pasos_corr  
d_s["Step_corr"] = Long_corr  
d_s["temper"] = tempers
d_s["Anchors"] = Anchors
file_name = "Droso_50reps_longer.pickle"
with open(file_name, "wb") as file:  # "wb" means "write in binary mode"
    pickle.dump(d_s, file)

In [ ]:
P_inv_t[-1,:,len(Anchors):len(Anchors)+88]

In [ ]:
P_inv_ini_t[-1,:,len(Anchors):len(Anchors)+21]

In [ ]:
P_t[-1,:,len(Anchors):]

In [ ]:
40*20/60

In [ ]:
8421/3600

In [ ]:
P_store.shape

In [ ]:
Pasos_store